<a href="https://colab.research.google.com/github/Zain1796/Lucky-Cement-AI-System/blob/main/NER_by_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.27.0
!pip install tiktoken
!pip install torchaudio librosa transformers openpyxl pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.6 MB/s eta 0:00:00


In [ ]:
import openai
import pandas as pd
import json
import numpy as np
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Joyn_NewDataNLP/colab_notebooks/api_key.txt') as f:
    api_key = f.read().strip()

In [ ]:
openai.api_key = api_key

In [ ]:
def extract_cement_entities(transcription_text):

    prompt_template = """
    You are a Quality Control Specialist overseeing field operations. You will analyze a transcript of a conversation between a retailer and an inquirer in Urdu, related to Pakistan’s cement market. Your task is to extract specific named entities.

    The conversation will mention various cement brands, focusing on details such as landed price, retail price, margins, sale quantity, stock, supplier, and payment methods. Extract the following information for the listed cement brands: 'DG', 'Lucky', 'Thatta', 'PakLand', 'Power', 'Falcon'. Any other brand should be categorized as 'Other'.

    For each brand, extract these details:
    - 'Landed': The rate at which cement is delivered to the outlet, including transport costs. For example, if the speaker askes "کتنے میں پڑا ہے ","پہنچ کتنا ہے ","کتنے میں آ رہا ہے  ","دے رہا ہے ","دیتے ہیں " and "لےنڈڈ ".
    - 'Retail': The rate at which the retailer is selling the cement brand. For example, if the speaker asks "کیا ریٹ چل رہا ہے ", and the response is "1030," this should be captured as the 'Retail' price for that brand.
    - 'Margin': This refers to the margin or savings per sale that the outlet earns. Include this only if there is an explicit mention in the conversation.
    - 'Sale Quantity': The total units sold by the outlet, expressed in either tons or bags.For example "کتنی کی چال ہے " and the responce is 40 this should be capture is 'Sale Quantity' of that brand.
    - 'Stock': The total available stock at the outlet in terms of bags(بوری) if there is any other quantity mentione pick that with the value.For example if the speaker asks "کتنا پڑا ہوا ہے " and the response is "بوری پڑی ہے 40",this should be captured as the 'Stock' for that brand.
    - 'Supplier': The dealer or supplier providing the cement to the retailer/outler or from the whole saler the retailer is buying the brands stock from.
    - 'Payment_Method': The payment method is the way at which funds is transfered or transiction has been occured, such as cash, check, credit (in case it mentions "udhaar" it means credit).


    Some things to keep in mind:
    - You have to always capture the updated/corrected values in the conversation. For example, if the speaker askes "1020 mi a raha hay?" and the response is "Nhi 1040" so 1040 should be captured as the 'Landed' price for that brand. Basically you should seek the confirmed value when there are two values for the same entity.
    - Any other brands not listed are grouped under 'Other'.
    - Differntiae between 'Landed' and 'Retail'.For example if the response is "20 30" it can be either 'Landed' or 'Retail' and capture the updated one.

    Provide the output in the following JSON format as a dictionary (do not mention the term `json` in the output):

    {{
        'DG': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'Lucky': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'Thatta': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'PakLand': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'Power': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'Falcon': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'Other': {{'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}},
        'Payment_Method': ''
    }}

    Following the dictionary, provide Qualitative anlysis as a market expert in a text each qualitative analysis against each topic should be in two lines.
    You are also to provide a brief analysis from Sales point of view of the overall transcript for the management to get a summary view if needed.A summary of the sales perspective to give management an overview of market trends
    based on the mentioned brands. If the transcript is too short and does not contain enough context, Write "Nothing useful in transcript".

    You will repeatedly see '#' in the transcript text, it is an identifier for speaker changed. As it's a conversation between two people speaker is changed as one asks question and the other answers and vice versa.

    The text is in Urdu with numbers in normal english digits, and you must pick the relative values and names as is when writing in the dict.

    Note: The result should only have 2 parts: first part should only be dictionary as explained and second part should be qualitative analysis.
"""


    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt_template},
                {"role": "user", "content": transcription_text}
            ],
            max_tokens=2000,
            temperature=0.01,
        )

        result =  response['choices'][0]['message']['content']

        result_parts = result.split("\n\n", 1)  # Split by two newlines

        # The first part is the dictionary (entities), convert it to a Python dict
        entities_dict_str = result_parts[0].strip()  # The first part should be the dictionary in string format

        try:
            # Use ast.literal_eval to convert the string representation of the dictionary into an actual dict
            entities_dict = ast.literal_eval(entities_dict_str)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing dictionary from response: {e}")
            entities_dict = {}

        summary = result_parts[1].strip() if len(result_parts) > 1 else ""

        return entities_dict, summary
    except openai.error.InvalidRequestError as e:
        print(f"Invalid request error: {e}")
        return None
    except openai.error.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return None

In [ ]:
tp = " یار بھائی یار بھائی  شکر ہم کو شکر ہم کو لگاے ہیں کن بونیہ لگاے ہیں الله معاف کرے کس کو مجھے 2 3 ہفتے سے گھر پے تھا الله کا شکر بس # بیماری بہت ہوگی یار # الله معاف کرے اسی وہ باہر ہے بچے گود ہے کچھ نہیں دیکھ رہے ہیں 2 2 3 3 ہفتہ گھر پے بھی اٹھا رہے ہیں # یہ یہ کچھ ہی اسکے # یار یہ ایک مچھا رہے ہیں مچھد سے مدے دیا ہے ڈینگی چکن گونی ہے اس کے ٹیقری میں بیماری الگ رہی ہے اس پر بھرے بھرے ہیں # اچھی نہیں # ڈینگی وغیرہ جو ہے اس سیل پنجاب کی طرف اس طرف ہے تو یہ سب سے زیادہ جو ہے چکن گونی ہے اچھی بخاریہ 3 3 میں اتار دیتا ہے وہ چلو ڈاکٹر میڈیکلی ہوجاتا ہے اس کا عل اس کا علاق نہیں ہے کاموں کا در ترلوں میں در نہیں ہے # یہ ڈم درو بھی یہ شفاہ اس سے ہوجاتا ہے الگ بات تو نہیں دائے گا  لیکن اس سے کام کیا کر رہا ہے کام ہلی ہیں سب چلو # میں نے کہا مزدور کو نہیں دے مزدور تو وہ ہے نا جو روز کمانا روز کھانے والا ہے اس کو نہیں دے ایک دو بندوں کو ہوا تھا سیمنٹ والاقے میں اس نے پتا نہیں کوئی میرے علاقے میں دکان میں کی مہینے کوئی بوری ٹم کرتا ہوا دے رہا ہے مجھے پتا نہیں اس نے بولا یار میرے مزدور گر دیتے ہیں ماشاللہ بوری تھی دوسرے دن کھڑے ہوگئے میں اس کے پاس بوری مجھے اس سے تھوڑا فرق آیا ماشاللہ الله کے پاس کلامی چیز نے تم کیم ہے # الله پر کچھ پاس دیتا ہے # زندگی بھی تھوڑا لیٹ ہوگیا دیتا ہے میں نے ایک دو گولیاں خائی مجھے بھی تھوڑا فرق آیا بیچا تھا # تھوڑا کیا ہے # واللہ کا شکر ہے مزدوروں نے بھی کھائے ہیں الله نے کرم کیا رہا ہے اور آپ سنائیں شکر کا کام وام پر سچل رہا ہے # گزارا یار کام وام # ڈی جی ماشاللہ زیادہ رکھا ہوا ہے # سیمنٹ میں کم آرہے کی نہیں ہے کم ہو رہے کم ہو رہے نہیں # نہیں کم کہاں سے وہ جب بکے نہیں رہا سریا ریٹ جو ہے صحیح ہے مگل کا کیا ریٹ آرہا ہے # مگل کا کیا ریٹ آرہا ہے # کیا حال میں ہیں # کتنے ادھر کراچی میں ہیں اب لوگوں کو # مگل تو نہیں آرہا فرصت ہوگیا ہے # منزہ # منزہ وغیرہ وہ ہے یہ 30 پے 25 میں 200 # یہ ریٹ یعنی بندے کو چاہیے تھا # ریٹ مانگ رہا تھا # اچھا نہیں آرہے لوگوں نہیں منگا رہے مطلب لوگوں کو بھی اپنا خراب ہوتا ہے پتا نہیں ہے # اپنا کوئی چاہیے # پہلے رکھو میں ہم تو پتا نہیں ہے روپے 7 روپے دے رہے مطلب ہے لوگوں کو بھی وامران سر خراب ہوگیا ہے پتا نہیں ہے #آپ نے کچھ کامیاب پہلے رکھوں میں ہم تو بندے پے دے رہے ہیں اوس کا ہے موحس کا ہے #موحس کیا چل رہا ہے# موحس کا سیمنٹ کا ہے اور 65 میں خالی کیا اور بچ میں بیچ رہے ہیں #وچاکے کچھ پای ہے #ہاں گاڑی مالی سب کرے ہے #نہیں کرے ہوگا #چلو کلو کرایا کتنوں کا ہے# کرایا کتنوں کا ہے# کرایا جیسے کرایا کرایا # اتنے کے حساب ہوتا ہے نا کرایا کتنے سے کتنے کا کرایا ہوتا ہے # 1200  تک تو یہاں قریب پر مہمار کے اندر ہے آپ کی پاس ابھی مگل پڑا ہوا ہے # مگل ہے اچھا اسکی ہے # ابھی پتا نہیں ہے # علی خان سے چل رہا ہے # اور کیا چلا رہا ہے # پاک لینڈ ہے پاک لینڈ کس سے لے رہا ہے # پاور# نیا ڈیلر آیا ہے # نیا نہیں ہے پاور نہیں ہے بھی یہاں پے # ریٹ اچھی چل رہا ہے # نہیں کام مہنگا یار # لکی تو 1100 میں دے رہے ہیں نا # نہیں آپ کو بھی # 70 میں # 70 میں اور ڈی جی # ڈی جی کم سے دے رہے ہیں پاور لکی برابر ہے پاور لکی برابر ہے اور سمان کرو پاک لینڈ کا ریٹ اوپر ہوا ہے 30  40  میں دے رہے ہیں # 15  # 1215  اچھا تو کھاتا چل رہا ہے اس کے ساتھ # ہاں کھاتا ہے # ڈی جی والے کے ساتھ بھی کھاتا ہے # سب سب # اور کھاتے کے ساتھ # ڈی جی والے کچھ میں یہ کھاتے ہیں # ابھی بہت کم ہو جائے گا # اور پتا نہیں کہ یار اور پتا نہیں کہ # کشھ میں نہیں اٹھایا # نہیں اٹھایا # حالات کل بہت # بہت بہت یار بیٹھ گیا پتا نہیں # بہت بیٹھ گیا بہت صحیح باقی تو بہت معاملہ سخت خراب ہے # کسا ہے بیٹھے کام ہو اسکا # "

In [ ]:
ner_result,summary = extract_cement_entities(tp)

In [ ]:
tp

' یار بھائی یار بھائی  شکر ہم کو شکر ہم کو لگاے ہیں کن بونیہ لگاے ہیں الله معاف کرے کس کو مجھے 2 3 ہفتے سے گھر پے تھا الله کا شکر بس # بیماری بہت ہوگی یار # الله معاف کرے اسی وہ باہر ہے بچے گود ہے کچھ نہیں دیکھ رہے ہیں 2 2 3 3 ہفتہ گھر پے بھی اٹھا رہے ہیں # یہ یہ کچھ ہی اسکے # یار یہ ایک مچھا رہے ہیں مچھد سے مدے دیا ہے ڈینگی چکن گونی ہے اس کے ٹیقری میں بیماری الگ رہی ہے اس پر بھرے بھرے ہیں # اچھی نہیں # ڈینگی وغیرہ جو ہے اس سیل پنجاب کی طرف اس طرف ہے تو یہ سب سے زیادہ جو ہے چکن گونی ہے اچھی بخاریہ 3 3 میں اتار دیتا ہے وہ چلو ڈاکٹر میڈیکلی ہوجاتا ہے اس کا عل اس کا علاق نہیں ہے کاموں کا در ترلوں میں در نہیں ہے # یہ ڈم درو بھی یہ شفاہ اس سے ہوجاتا ہے الگ بات تو نہیں دائے گا  لیکن اس سے کام کیا کر رہا ہے کام ہلی ہیں سب چلو # میں نے کہا مزدور کو نہیں دے مزدور تو وہ ہے نا جو روز کمانا روز کھانے والا ہے اس کو نہیں دے ایک دو بندوں کو ہوا تھا سیمنٹ والاقے میں اس نے پتا نہیں کوئی میرے علاقے میں دکان میں کی مہینے کوئی بوری ٹم کرتا ہوا دے رہا ہے مجھے پتا نہیں اس نے بولا یار میرے مزدور گر دیتے ہیں 

In [ ]:
ner_result

{'DG': {'Landed': '70',
  'Retail': '1100',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'Lucky': {'Landed': '',
  'Retail': '1100',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'Thatta': {'Landed': '',
  'Retail': '',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'PakLand': {'Landed': '1215',
  'Retail': '',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'Power': {'Landed': '',
  'Retail': '',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'Falcon': {'Landed': '',
  'Retail': '',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'Other': {'Landed': '',
  'Retail': '',
  'Margin': '',
  'Sale Quantity': '',
  'Stock': '',
  'Supplier': ''},
 'Payment_Method': ''}

In [ ]:
from typing_extensions import Text
p = Text(ner_result)

In [ ]:
p

"{'DG': {'Landed': '70', 'Retail': '1100', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'Lucky': {'Landed': '', 'Retail': '1100', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'Thatta': {'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'PakLand': {'Landed': '1215', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'Power': {'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'Falcon': {'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'Other': {'Landed': '', 'Retail': '', 'Margin': '', 'Sale Quantity': '', 'Stock': '', 'Supplier': ''}, 'Payment_Method': ''}"